In [ ]:
from google.colab import files
uploaded = files.upload()

Saving mfcc_train.csv to mfcc_train (3).csv


In [ ]:
import io
import pandas as pd

mfcc_train = pd.read_csv(io.StringIO(uploaded['mfcc_train.csv'].decode('utf-8')))
mfcc_train.drop(['Unnamed: 0'], axis = 1, inplace = True)
mfcc_train = mfcc_train.reset_index(drop=True)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving mfcc_test.csv to mfcc_test (4).csv


In [ ]:
import io
import pandas as pd

mfcc_test = pd.read_csv(io.StringIO(uploaded['mfcc_test.csv'].decode('utf-8')))
mfcc_test.drop(['Unnamed: 0'], axis = 1, inplace = True)
mfcc_test= mfcc_test.reset_index(drop=True)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving seol_train.csv to seol_train (3).csv


In [ ]:
import io
import pandas as pd

seol_train = pd.read_csv(io.StringIO(uploaded['seol_train.csv'].decode('utf-8')))
di_train = seol_train.iloc[:,3]
seol_train = seol_train.drop(['Diagnosis'],axis =1)
seol_train.drop(['Unnamed: 0'], axis = 1, inplace = True)
seol_train = seol_train.reset_index(drop=True)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving seol_test.csv to seol_test (3).csv


In [ ]:
import io
import pandas as pd

seol_test = pd.read_csv(io.StringIO(uploaded['seol_test.csv'].decode('utf-8')))
di_test = seol_test.iloc[:,3]
seol_test = seol_test.drop(['Diagnosis'],axis =1)
seol_test.drop(['Unnamed: 0'], axis = 1, inplace = True)
seol_test = seol_test.reset_index(drop=True)

In [ ]:
!pip install boruta

In [ ]:
import math
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from boruta import BorutaPy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV
import librosa
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

데이터 정규화및 보루타 사용

In [ ]:
import random
random.seed(42)

seol = seol_train.values
di = di_train.values
di2 = di_test.values
mfcc = mfcc_train.values
mfcc2 = mfcc_test.values
# 데이터 정규화
sc = StandardScaler()

mfcc = sc.fit_transform(mfcc)
mfcc = mfcc.reshape(mfcc.shape[0], -1)

mfcc2 = sc.fit_transform(mfcc2)
mfcc2 = mfcc2.reshape(mfcc2.shape[0], -1)

rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
# Boruta 알고리즘으로 변수 선택
boruta = BorutaPy(rf, n_estimators='auto', verbose=0, random_state=42,perc = 50)
boruta.fit(mfcc,di)

# 선택된 변수만 추출
selected_features = mfcc[:, boruta.support_]

X_train_b = mfcc[:, boruta.support_]
X_test_b = mfcc2[:, boruta.support_]

mfcc smote 사용

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, confusion_matrix

smote = SMOTE(random_state=42)
X_train_s, y_train = smote.fit_resample(X_train_b, di)

mlp 코드 적용

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(17,17,17,17,17), activation='relu', solver='adam', alpha=0.1, batch_size=22,
              learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=150, random_state=42)

# 모델 학습
mlp.fit(X_train_s, y_train)
# 예측
y_pred = mlp.predict(X_test_b)

acc = accuracy_score(di2, y_pred)
mlps = mlp.predict(X_test_b)
prob = mlp.predict_proba(X_test_b)

seol smote 적용

In [ ]:
X_train2 = seol_train.values
y_train2 = di_train.values

X_test2 = seol_test.values
y_test2 = di_test.values

X_train2_s, y_train2 = smote.fit_resample(X_train2, y_train2)

xgb 코드 적용

In [ ]:
xgb = XGBClassifier(n_estimators=100,max_depth =5,subsample=1,
              reg_alpha=0.1,reg_lambda=0.3,learning_rate =0.5,colsample_bytree = 1,min_child_weight=2,gamma = 1)
fit = xgb.fit(X_train2_s, y_train2)
preds1 = fit.predict(X_test2)
acc2 = accuracy_score(di2, preds1)
xgbs = xgb.predict(X_test2)
prob2 = xgb.predict_proba(X_test2)

In [ ]:
print(acc,acc2)

0.42857142857142855 0.7142857142857143


In [ ]:
proba1 = pd.DataFrame(prob)
proba2 = pd.DataFrame(prob2)
result = pd.concat([proba1,proba2], axis=1)

In [ ]:
proba1 = pd.DataFrame(prob)
proba2 = pd.DataFrame(prob2)
result = pd.concat([proba1,proba2], axis=1)
rf = RandomForestClassifier(n_estimators=10, max_depth =2,random_state=42)
rf.fit(result, di2)

final_preds = rf.predict(result)
acc = accuracy_score(di2, final_preds)
print(acc)

0.7857142857142857


In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# 혼동 행렬 (Confusion Matrix)을 계산합니다.
confusion = confusion_matrix(di2, final_preds)

# 정밀도 (Precision)를 계산합니다.
precision = precision_score(di2, final_preds)

# 재현율 (Recall)을 계산합니다.
recall = recall_score(di2, final_preds)

# F1 스코어를 계산합니다.
f1 = f1_score(di2, final_preds)

# 특이도 (Specificity)를 계산합니다.
# 특이도는 다음과 같이 계산할 수 있습니다:
# 특이도 = (TN) / (TN + FP)
tn, fp, fn, tp = confusion.ravel()
specificity = tn / (tn + fp)

# 결과 출력
print("혼동 행렬:")
print(confusion)
print("정밀도 (Precision):", precision)
print("재현율 (Recall):", recall)
print("F1 스코어:", f1)
print("특이도 (Specificity):", specificity)

혼동 행렬:
[[57  4]
 [14  9]]
정밀도 (Precision): 0.6923076923076923
재현율 (Recall): 0.391304347826087
F1 스코어: 0.5
특이도 (Specificity): 0.9344262295081968


smote-enn

In [ ]:
from imblearn.combine import SMOTEENN
from sklearn.metrics import accuracy_score, confusion_matrix

smote_enn = SMOTEENN(random_state=42)

X_train_b
X_test_b
X_train2 = seol_train.values
X_test2 = seol_test.values
combined_data = np.concatenate((X_train_b, X_train2),axis=1)
combined_data2 = np.concatenate((X_test_b, X_test2),axis=1)

com_se, y_train =smote_enn.fit_resample(combined_data, di)

In [ ]:
X_train_se = com_se[:,:X_train_b.shape[1]]
X_train2_se = com_se[:,X_train_b.shape[1]:]

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(17,17,17,17,17), activation='relu', solver='adam', alpha=0.1, batch_size=22,
              learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=150, random_state=42)

# 모델 학습
mlp.fit(X_train_se, y_train)
# 예측
y_pred = mlp.predict(X_test_b)

acc = accuracy_score(di2, y_pred)
mlps = mlp.predict(X_test_b)
prob = mlp.predict_proba(X_test_b)

In [ ]:
xgb = XGBClassifier(n_estimators=100,max_depth =5,subsample=1,
              reg_alpha=0.1,reg_lambda=0.3,learning_rate =0.5,colsample_bytree = 1,min_child_weight=2,gamma = 1)
fit = xgb.fit(X_train2_se, y_train)
preds1 = fit.predict(X_test2)
acc2 = accuracy_score(di2, preds1)
xgbs = xgb.predict(X_test2)
prob2 = xgb.predict_proba(X_test2)

In [ ]:
y_test = di2

In [ ]:
proba1 = pd.DataFrame(prob)
proba2 = pd.DataFrame(prob2)
result = pd.concat([proba1,proba2], axis=1)
rf = RandomForestClassifier(n_estimators=10, max_depth =2,random_state=42)
rf.fit(result, y_test)

final_preds = rf.predict(result)
acc = accuracy_score(y_test, final_preds)
print(acc)

0.7857142857142857


In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# 혼동 행렬 (Confusion Matrix)을 계산합니다.
confusion = confusion_matrix(y_test, final_preds)

# 정밀도 (Precision)를 계산합니다.
precision = precision_score(y_test, final_preds)

# 재현율 (Recall)을 계산합니다.
recall = recall_score(y_test, final_preds)

# F1 스코어를 계산합니다.
f1 = f1_score(y_test, final_preds)

# 특이도 (Specificity)를 계산합니다.
# 특이도는 다음과 같이 계산할 수 있습니다:
# 특이도 = (TN) / (TN + FP)
tn, fp, fn, tp = confusion.ravel()
specificity = tn / (tn + fp)

# 결과 출력
print("혼동 행렬:")
print(confusion)
print("정밀도 (Precision):", precision)
print("재현율 (Recall):", recall)
print("F1 스코어:", f1)
print("특이도 (Specificity):", specificity)

혼동 행렬:
[[53  8]
 [10 13]]
정밀도 (Precision): 0.6190476190476191
재현율 (Recall): 0.5652173913043478
F1 스코어: 0.5909090909090909
특이도 (Specificity): 0.8688524590163934


ADASYN 적용

In [ ]:
from imblearn.over_sampling import ADASYN
from sklearn.metrics import accuracy_score, confusion_matrix

adasyn = ADASYN(random_state=42)

X_train_b
X_test_b
X_train2 = seol_train.values
X_test2 = seol_test.values
combined_data = np.concatenate((X_train_b, X_train2),axis=1)
combined_data2 = np.concatenate((X_test_b, X_test2),axis=1)

com_a, y_train =adasyn.fit_resample(combined_data, di)

In [ ]:
X_train_a = com_a[:,:X_train_b.shape[1]]
X_train2_a = com_a[:,X_train_b.shape[1]:]

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(17,17,17,17,17), activation='relu', solver='adam', alpha=0.1, batch_size=22,
              learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=150, random_state=42)

# 모델 학습
mlp.fit(X_train_a, y_train)
# 예측
y_pred = mlp.predict(X_test_b)

acc1 = accuracy_score(di2, y_pred)
mlps = mlp.predict(X_test_b)
prob = mlp.predict_proba(X_test_b)

In [ ]:
xgb = XGBClassifier(n_estimators=100,max_depth =5,subsample=1,
              reg_alpha=0.1,reg_lambda=0.3,learning_rate =0.5,colsample_bytree = 1,min_child_weight=2,gamma = 1)
fit = xgb.fit(X_train2_a, y_train)
preds1 = fit.predict(X_test2)
acc2 = accuracy_score(, preds1)
xgbs = xgb.predict(X_test2)
prob2 = xgb.predict_proba(X_test2)

In [ ]:
proba1 = pd.DataFrame(prob)
proba2 = pd.DataFrame(prob2)
result = pd.concat([proba1,proba2], axis=1)
rf = RandomForestClassifier(n_estimators=10, max_depth =2,random_state=42)
rf.fit(result, y_test)

final_preds = rf.predict(result)
acc = accuracy_score(y_test, final_preds)
print(acc)

0.75


In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# 혼동 행렬 (Confusion Matrix)을 계산합니다.
confusion = confusion_matrix(y_test, final_preds)

# 정밀도 (Precision)를 계산합니다.
precision = precision_score(y_test, final_preds)

# 재현율 (Recall)을 계산합니다.
recall = recall_score(y_test, final_preds)

# F1 스코어를 계산합니다.
f1 = f1_score(y_test, final_preds)

# 특이도 (Specificity)를 계산합니다.
# 특이도는 다음과 같이 계산할 수 있습니다:
# 특이도 = (TN) / (TN + FP)
tn, fp, fn, tp = confusion.ravel()
specificity = tn / (tn + fp)

# 결과 출력
print("혼동 행렬:")
print(confusion)
print("정밀도 (Precision):", precision)
print("재현율 (Recall):", recall)
print("F1 스코어:", f1)
print("특이도 (Specificity):", specificity)

혼동 행렬:
[[18 10]
 [ 4 24]]
정밀도 (Precision): 0.7058823529411765
재현율 (Recall): 0.8571428571428571
F1 스코어: 0.7741935483870968
특이도 (Specificity): 0.6428571428571429


MWSMOTE

In [ ]:
!pip install smote_variants

In [ ]:
import smote_variants as sv
oversampler = sv.MWMOTE(random_state=42)

X_train_b
X_test_b
X_train2 = seol_train.values
X_test2 = seol_test.values
combined_data = np.concatenate((X_train_b, X_train2),axis=1)
combined_data2 = np.concatenate((X_test_b, X_test2),axis=1)

com_mw, y_train = oversampler.sample(combined_data, di)

2023-11-05 16:27:27,741:INFO:MWMOTE: Running sampling via ('MWMOTE', "{'proportion': 1.0, 'k1': 5, 'k2': 5, 'k3': 5, 'M': 10, 'cf_th': 5.0, 'cmax': 10.0, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 42, 'class_name': 'MWMOTE'}")
INFO:smote_variants:MWMOTE: Running sampling via ('MWMOTE', "{'proportion': 1.0, 'k1': 5, 'k2': 5, 'k3': 5, 'M': 10, 'cf_th': 5.0, 'cmax': 10.0, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 42, 'class_name': 'MWMOTE'}")
2023-11-05 16:27:27,749:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
INFO:smote_variants:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2023-11-05 16:27:27,754:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
INFO:smote_variants:NearestNeighbors

In [ ]:
X_train_mw = com_mw[:,:X_train_b.shape[1]]
X_train2_mw = com_mw[:,X_train_b.shape[1]:]

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(17,17,17,17,17), activation='relu', solver='adam', alpha=0.1, batch_size=22,
              learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=150, random_state=42)

# 모델 학습
mlp.fit(X_train_mw, y_train)
# 예측
y_pred = mlp.predict(X_test_b)

acc1 = accuracy_score(di2, y_pred)
mlps = mlp.predict(X_test_b)
prob = mlp.predict_proba(X_test_b)

In [ ]:
xgb = XGBClassifier(n_estimators=100,max_depth =5,subsample=1,
              reg_alpha=0.1,reg_lambda=0.3,learning_rate =0.5,colsample_bytree = 1,min_child_weight=2,gamma = 1)
fit = xgb.fit(X_train2_mw, y_train)
preds1 = fit.predict(X_test2)
acc2 = accuracy_score(di2, preds1)
xgbs = xgb.predict(X_test2)
prob2 = xgb.predict_proba(X_test2)

In [ ]:
proba1 = pd.DataFrame(prob)
proba2 = pd.DataFrame(prob2)
result = pd.concat([proba1,proba2], axis=1)
rf = RandomForestClassifier(n_estimators=10, max_depth =2,random_state=42)
rf.fit(result, di2)

final_preds = rf.predict(result)
acc = accuracy_score(di2, final_preds)
print(acc)

0.8452380952380952


In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# 혼동 행렬 (Confusion Matrix)을 계산합니다.
confusion = confusion_matrix(di2, final_preds)

# 정밀도 (Precision)를 계산합니다.
precision = precision_score(di2, final_preds)

# 재현율 (Recall)을 계산합니다.
recall = recall_score(di2, final_preds)

# F1 스코어를 계산합니다.
f1 = f1_score(di2, final_preds)

# 특이도 (Specificity)를 계산합니다.
# 특이도는 다음과 같이 계산할 수 있습니다:
# 특이도 = (TN) / (TN + FP)
tn, fp, fn, tp = confusion.ravel()
specificity = tn / (tn + fp)

# 결과 출력
print("혼동 행렬:")
print(confusion)
print("정밀도 (Precision):", precision)
print("재현율 (Recall):", recall)
print("F1 스코어:", f1)
print("특이도 (Specificity):", specificity)

혼동 행렬:
[[55  6]
 [ 7 16]]
정밀도 (Precision): 0.7272727272727273
재현율 (Recall): 0.6956521739130435
F1 스코어: 0.711111111111111
특이도 (Specificity): 0.9016393442622951
